In [235]:
import os
import json
import requests
import re, html, unicodedata
import pandas as pd
from sqlalchemy import create_engine,text
import requests
import time
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time
from bs4.element import Tag


In [236]:
# Credenciales de la Base de Datos ANTIGUA (saingo_v2)
DB_OLD_USER = "root"
DB_OLD_PASS = ""
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA (saingo_v5)
DB_NEW_USER = "root"
DB_NEW_PASS = ""
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"

##id mapping pero no la usare

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")

In [237]:
def create_db_engine(user, password, host, dbname):
    connection_string = f"mysql+pymysql://{user}:{password}@{host}/{dbname}?charset=utf8mb4"
    return create_engine(connection_string)

def fetch_old_gallery_data():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT id, title, excerpt, slug, body, seo_title, meta_description, tags, campaign_id FROM blog_posts" ## selecciona los titulos y id de blog posts

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

def fetch_new_gallery_data():
    engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    query = "SELECT id,title FROM reports" ## selecciona los titulos y id de blog posts

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df




campaing_id=17

db=fetch_old_gallery_data()
db_new=fetch_new_gallery_data()

def comprobar(title):
    proof=False
    tamano=len(db_new)
    for j in range(0,tamano):
        json=db_new.iloc[j]
        cadena=json.get("title").split('"')
        final=cadena[3]
        #vector=cadena.split(",")
        #conversion=vector[0][7:len(vector[0])-1]
        #final=conversion.strip('"')
        if(final==title):
            proof=True
            return proof
    return proof


def coincidencia(title,db_new):
   
 
    for i in range(0,len(db_new)):
        title_new=db_new.iloc[i].get('title')
        cadena=title_new.split('"')
        final=cadena[3]
        if(final==title):
            id_new=db_new.iloc[i].get('id')
            return id_new

        

    




    return 200



def busqueda_imagen(body):
    ext=""
    soup=BeautifulSoup(body,'html.parser')
    imagenes=soup.find_all("img")
    print(imagenes)
    if(imagenes):
        img=imagenes[0]
        vector=img.get("src").split('/')
        ext=vector[len(vector)-1]




    return ext


def subir_foto(ext, id_new, espera=2, tiempo_maximo=5):
    """
    Hace una petición GET a la URL generada con ext e id_new.
    
    espera: segundos de descanso después de cada petición
    tiempo_maximo: segundos máximos que esperamos respuesta del servidor
    """
    url = f"http://127.0.0.1:8000/reports/featured-image/{ext}/{id_new}"

    try:
        response = requests.get(url, timeout=tiempo_maximo)  # tiempo máximo de espera
        print(url)

        if response.status_code == 200:
            print("Petición exitosa")
            print(response.text)
        else:
            print(f"Error {response.status_code}")
    except requests.exceptions.Timeout:
        print(f"Tiempo de espera agotado ({tiempo_maximo}s) para {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error en la petición: {e}")

    time.sleep(espera)  # descanso antes de la siguiente petición


for i in range(0,len(db)):
    if pd.notna(db.iloc[i].get('campaign_id')):
        if(int(db.iloc[i].get('campaign_id'))==campaing_id):
            id=db.iloc[i].get('id')
            title=db.iloc[i].get('title')
            body=db.iloc[i].get('body')
            ext=busqueda_imagen(body)
            id_new=coincidencia(title,db_new)
            subir_foto(ext, id_new, espera=3, tiempo_maximo=10)
            #print(ext)
            
            











                

   
           



[<img alt="SAI (South American Initiative), a dedicated non-profit in Venezuela, ensures rescued monkeys receive proper nourishment and compassionate care. This image captures a volunteer in a white t-shirt labeled “South American Initiative,” offering food through a green metal fence to three eager monkeys. Their outstretched hands and focused expressions highlight their trust and reliance on structured care. The fenced enclosure in the background reinforces the secure habitat designed for their well-being. Thanks to the dedication of volunteers and the generosity of donors, SAI continues to support rescued wildlife in Venezuela, providing them with essential nourishment and protection to thrive." class="lazyloaded" data-src="https://www.globalgiving.org/pfil/39741/IMG_4993_Large.JPG" height="600" src="https://sai.ngo/public/storage/blog_posts/April2025/IMG_4993_Large.png" style="display: block; margin-left: auto; margin-right: auto;" title="SAI Volunteers Provide Nutritional Support 